# Decompositions and least squares


In this lecture, we look at several factorizations of a matrix. For a square or rectangular matrix $A ∈ ℝ^{m × n}$ with $m ≥ n$, we consider:
1. The _QR decomposition_
$$
A = \underbrace{Q}_{m × m} \underbrace{R}_{m × n} = \begin{bmatrix} 𝐪_1 | \cdots | 𝐪_m \end{bmatrix} \begin{bmatrix} × & \cdots & × \\ & \ddots & \vdots \\ && × \\ &&0 \\ &&\vdots \\ && 0 \end{bmatrix} 
$$
where $Q$ is orthogonal ($Q^⊤Q = I$, $𝐪_j ∈ ℝ^m$) and $R$ is _right triangular_.

2. The _reduced QR decomposition_
$$
A = \underbrace{Q̂}_{m × n} \underbrace{R̂}_{m × m} = \begin{bmatrix} 𝐪_1 | \cdots | 𝐪_n \end{bmatrix} \begin{bmatrix} × & \cdots & × \\ & \ddots & \vdots \\ && ×  \end{bmatrix} 
$$
where $Q$ has orthogonal columns ($Q^⊤Q = I$, $𝐪_j ∈ ℝ^m$) 

For a square matrix we consider the _PLU decomposition_:
$$
A = P^⊤ LU
$$
where $P$ is a permutation matrix, $L$ is lower triangular and $U$ is upper triangular.

Finally, for a square, _symmetric positive definite_ ($𝐱^⊤ A 𝐱 > 0$ for all $𝐱 ∈ ℝ^n$) 
matrix we consider the _Cholesky decomposition_:
$$
A = L L^⊤
$$

The importance of these decomposition for square matrices is that their component pieces are easy to invert on a computer:
$$
\begin{align*}
A = P^⊤ LU &\Rightarrow\qquad A^{-1}𝐛 = U^{-1} L^{-1} P 𝐛 \\
A = QR &\Rightarrow\qquad A^{-1}𝐛 = R^{-1} Q^\top 𝐛 \\
A = L L^⊤ &\Rightarrow\qquad A^{-1}𝐛 = L^{-⊤} L^{-1} 𝐛
\end{align*}
$$
and we saw last lecture that triangular and orthogonal matrices are easy to invert when applied to a vector $𝐛$.
For rectangular matrices we will see that they lead to efficient solutions to the _least squares problem_: find
$𝐱$ that minimizes the 2-norm
$$
\| A 𝐱 - 𝐛 \|.
$$

In this lecture we discuss the followng:

1. QR,  Reduced QR, and least squares: We discuss the QR decomposition and its usage in solving least squares problems.
We discuss computation of the Reduced QR decomposition using Gram–Schmidt, and the Full QR decomposition using 
Householder reflections.
2. PLU decomposition: we discuss how the LU decomposition can be computed using Gaussian elimination, and the computation of
the PLU decomposition via Gaussian elimination with pivoting.
3. Cholesky decomposition: we introduce symmetric positive definite matrices and show that their LU decomposition can be re-interpreted
as a Cholesky decomposition.

In [1]:
using LinearAlgebra, Plots

## 1. QR, Reduced QR, and least squares

Here we consider rectangular matrices with more rows than columns.
A QR decomposition decomposes a matrix into an orthogonal matrix $Q$ times a right triangular matrix $R$. 
Note the QR decomposition contains within it the reduced QR decomposition:
$$
A = QR = \begin{bmatrix} Q̂ | 𝐪_{n+1} | ⋯ | 𝐪_m \end{bmatrix} \begin{bmatrix} R̂ \\  𝟎_{m-n × n} \end{bmatrix} = Q̂ R̂.
$$

### Relationship with least squares

We can use it to solve a least squares problem using the norm-preserving property (see PS3) of orthogonal matrices:
$$
\| A 𝐱 - 𝐛 \| = \| Q R 𝐱 - 𝐛 \| = \| R 𝐱 - Q^⊤ 𝐛 \| = \left \| 
\begin{bmatrix} R̂ \\ 𝟎_{m-n × n} \end{bmatrix} 𝐱 - \begin{bmatrix} Q̂^⊤ \\ 𝐪_{n+1}^⊤ \\ \vdots \\ 𝐪_m^⊤ \end{bmatrix}     𝐛 \right \|
$$
Now note that the rows $k > n$ are independent of $𝐱$ and are a fixed contribution. Thus to minimise this norm it suffices to
drop them and minimise:
$$
\| R̂ 𝐱 - Q̂^⊤ 𝐛 \|
$$
This norm is minimisable if it is attained. Provided the column rank of $A$ is full, $R̂$ will be invertible (Exercise: why is this?).
Thus we have the solution
$$
𝐱 = R̂^{-1} Q̂^⊤ 𝐛
$$

**Example (quadratic fit)** Suppose we want to find noisy data by a quadratic
$$
p(x) = a + bx + cx^2
$$
That is, we want to choose $a,b,c$ at data samples $x_1, \ldots, x_m$ so that the following is true:
$$
a + b x_k + c x_k^2 ≈ f_k
$$
where $f_k$ are given by data. We can reinterpret this as a least squares problem: minimise the norm
$$
\left\| \begin{bmatrix} 1 & x_1 & x_1^2 \\ \vdots & \vdots & \vdots \\ 1 & x_m & x_m^2 \end{bmatrix}
\begin{bmatrix} a \\ b \\ c \end{bmatrix} - \begin{bmatrix} f_1 \\ \vdots \\ f_m \end{bmatrix} \right \|
$$
We can solve this using the QR decomposition:

In [2]:
m,n = 100,3

x = range(0,1; length=m) # 100 points
f = 2 .+ x .+ 2x.^2 .+ 0.1 .* randn.() # Noisy quadratic

A = [ones(n) x x.^2] # 100 x 3 matrix
Q,R̂ = qr(A)
Q̂ = Q[:,1:n] # Q represents full orthogonal matrix so we take first 3 columns

a,b,c = R̂ \ Q̂'f

LoadError: ArgumentError: number of rows of each array must match (got (3, 100, 100))

We can visualise the fit:

In [3]:
p = x -> a + b*x + c*x^2

scatter(x, f; label="samples", legend=:bottomright)
plot!(x, p.(x); label="quadratic")

LoadError: UndefVarError: b not defined

Note that `\` with a rectangular system does the least squares by default:

In [4]:
A \ f

LoadError: UndefVarError: A not defined

### Gram–Schmidt and reduced QR


How do we compute the QR decomposition? We begin with a method
you may have seen before in another guise. Write
$$
A = \begin{bmatrix} 𝐚_1 | \dots | 𝐚_n \end{bmatrix}
$$
where $𝐚_k \in  ℝ^m$.
Note that the column span of the first $j$ columns of $A$
will be the same as the first $j$ columns of $Q̂$:
$$
\span{𝐚_1,\ldots,𝐚_j} = \span{𝐪_1,\ldots,𝐪_j}
$$
In other words: the columns of $Q̂$ are an orthogonal basis
of the column span of $A$.
To see this: note that since `R̂` is triangular we have:

In [5]:
A[:,1:j] = Q̂[:,1:j]*R̂[1:j,1:j]

LoadError: UndefVarError: j not defined

for all $j$, and the multiplication is taking linear combinations of
`Q̂[:,1:j]`. 
 
It is possible to find an orthogonal basis using the _Gram–Schmidt algorithm_,
We construct it via induction:
assume that
$$
\span{𝐚_1,\ldots,𝐚_{j-1}} = \span{𝐪_1,\ldots,𝐪_{j-1}}
$$
where $𝐪_1,\ldots,𝐪_{j-1}$ are orthogonal:
$$
𝐪_k^\top 𝐪_ℓ = δ_{kℓ} = \begin{cases} 1 & k == ℓ \\
                                            0 & \hbox{otherwise} \end{cases}.
$$
for $k,ℓ < 1$.
Define
$$
𝐯_j := 𝐚_j - \sum_{k=1}^{j-1} \underbrace{𝐪_k^\top 𝐚_j}_{v_{kj}} 𝐪_k
$$
so that for $k < j$
$$
𝐪_k^\top 𝐯_j = 𝐪_k^\top 𝐚_j - \sum_{k=1}^{j-1} \underbrace{𝐪_k^\top 𝐚_j}_{v_{kj}} 𝐪_k^\top 𝐪_k = 0.
$$
Then we define
$$
𝐪_j := {𝐯_j \over \|𝐯_j\|}.
$$
which sastisfies the same properties as the assumption.

We now reinterpret this construction as a reduced QR decomposition.
Define
$$
r_{jj} := {1 \over \|𝐯_j\|}, \qquad r_{kj} := {v_{kj} \over \|𝐯_j\|}
$$
Then rearrange the definition we have
$$
𝐚_j = \begin{bmatrix} 𝐪_1|\cdots|𝐪_j \end{bmatrix} \begin{bmatrix} r_{1j} \\ \vdots \\ r_{jj} \end{bmatrix}
$$
Thus
$$
\begin{bmatrix} 𝐚_1|\cdots|𝐚_j \end{bmatrix} r_{11} & \cdots & r_{1j} \\ & \ddots & \vdots \\ && r_{jj} \end{bmatrix}
$$
That is, we are computing the reduced QR decomposition column-by-column. Running this algorithm to $j = n$ completes the decomposition.

Unfortunately, the Gram–Schmidt algorithm is _unstable_: the rounding errors when implemented in floating point
accumulate in a disasterous way. This will be explored in the problem sheet.


### Householder reflections and QR

As an alternative, we will consider using Householder reflections to introduce zeros.
Thus, if Gram–Schmidt is a process of _triangular orthogonalisation_ (using triangular matrices
to orthogonalise), Householder reflections is a process of _orthogonal triangularisation_ 
(using orthogonal matrices to triangularise).

Consider multiplication by the Householder reflection corresponding to the first column,
that is, for
$$
Q_1 := Q_{𝐚_1}^{±,\rm H},
$$
consider
$$
Q_1 A = \begin{bmatrix} r_{11} & r_{12} & \cdots & r_{1n} \\ 
& 𝐚_2^1 & \cdots & 𝐚_n^1   \end{bmatrix}
$$
where 
$$
r_{1j} :=  (Q_1 𝐚_j)[1] \qquad \hbox{and} \qquad 𝐚_j^1 := (Q_1 𝐚_j)[2:m],
$$
noting $r_{11} = ±\|𝐚_1\|$ and all entries of $𝐚_1^1$ are zero (thus not included).
That is, we have made the first column triangular.

But now consider
$$
Q_2 := \begin{bmatrix} 1  \\ & Q_{𝐚_2^1}^{±,\rm H} \end{bmatrix}
$$
so that
$$
Q_2 Q_1A  \begin{bmatrix} r_{11} & r_{12} & \cdots & r_{1n} \\ 
    & r_{22} & r_{23} \cdots & r_{2n} \\
&& 𝐚_3^2 & \cdots & 𝐚_n^2   \end{bmatrix}
$$
where 
$$
r_{2j} :=  (Q_2 𝐚_j^1)[1] \qquad \hbox{and} \qquad 𝐚_j^2 := (Q_2 𝐚_j^1)[2:m-1]
$$
Thus the first two columns are triangular. 

The inductive construction is thus clear. If we define $𝐚_j^0 := 𝐚_j$ we
have the construction
$$
\begin{align*}
Q_j &:= \begin{bmatrix} I_{j-1 \times j-1}  \\ & Q_{𝐚_j^j}^{±,\rm H} \end{bmatrix} \\
𝐚_j^k &:= (Q_k 𝐚_j^{k-1})[2:m-k+1] \\
r_{kj} &:= (Q_k 𝐚_j^{k-1})[1]
\end{align*}
$$
Then
$$
Q_n \cdots Q_1 A = \underbrace{\begin{bmatrix} 
r_{11} & \cdots & r_{1n} \\ & \ddots & \vdots\\
                                        && r_{nn} \\&& 0 \\ && \vdots \\ && 0 \end{bmatrix}}_R
$$
i.e.
$$
A = \underbrace{Q_n \cdots Q_1}_Q R.
$$

The implementation is cleaner. We do a naive implementation here:

In [6]:
function householderreflection(x)
    y = copy(x)
    y[1] += sign(x[1])norm(x)
    w = y/norm(y)
    I - 2*w*w'
end

m,n = 7,5
A = randn(m, n)
R = copy(A)
Q = Matrix(1.0I, m, m)
for j = 1:n
    Qⱼ = householderreflection(R[j:end,j])
    R[j:end,:] = Qⱼ*R[j:end,:]
    Q[:,j:end] = Q[:,j:end]*Qⱼ
end
Q*R ≈ A

true

Note because we are forming a full matrix representation of each Householder
reflection this is a slow algorithm, taking $O(n^4)$ operations. The problem sheet
will consider a better implementation that takes $O(n^3)$ operations.


**Example** We will now do an example by hand. Consider the $4 \times 3$ matrix
$$
A = \begin{bmatrix} 
1 & 2 & 2 \\
1 & 1 & 2 \\
1 & 1 & 1 \\
1 & 1 & 1.
$$
For the first column we have
$$
Q_1 = I - {1 \over 6} \begin{bmatrix}3 \\ 1\\ 1\\ 1 \end{bmatrix} \begin{bmatrix}3 & 1& 1& 1 \end{bmatrix} = {1 \over 6} \begin{bmatrix} -3 & -3 & -3 & -3\\ -3 & 5 &-1 & -1 \\ -3 & -1 & 5 & -1 &-3 &-1 & -1 5 \end{bmatrix}
$$
and
$$
Q_1 A = {1 \over 6} \begin{bmatrix} -12 & -15 &  -18 \\  & -3 & 2 \\ &-3 &-4\\ -3 & -4 \end{bmatrix}
$$
Then 
$$
Q_2 = \begin{bmatrix} 1 \\ & I - 2
\end{bmatrix}
$$


## 2. PLU Decomposition

Just as Gram–Schmidt can be reinterpreted as a reduced QR decomposition,
Gaussian elimination with pivoting can be interpreted as a PLU decomposition.


### LU Decomposition

Before discussing pivoting, consider standard Gaussian elimation where one row-reduces
to introduce zeros column-by-column. We will mimick the computation of the QR decomposition
to view this as a _triangular triangularisation_.

Consider the matrix
$$
L_𝐯 = \begin{bmatrix} 1 \\ -{v_2 \over v_1} & 1 \\ \vdots &&\ddots \\
                -{v_n \over v_1}  &&& 1
                \end{bmatrix} = I -\left[{𝐯 \over v_1}  - 𝐞_1 \right] 𝐞_1^\top
$$
so that
$$
L_𝐯 𝐯 = 𝐯 - (𝐯 - v_1 𝐞_1) = v_1 𝐞_1.
$$
Note that this is immediately invertible:
$$
L_𝐯^{-1} = \begin{bmatrix} 1 \\ {v_2 \over v_1} & 1 \\ \vdots &&\ddots \\
                {v_n \over v_1}  &&& 1
                \end{bmatrix} = I + \left[{𝐯 \over v_1}  - 𝐞_1 \right] 𝐞_1^\top
$$


For $L_1 := L_{𝐚_1}$ we have 
$$
L_1 A =  \begin{bmatrix} u_{11} & u_{12} & \cdots & u_{1n} \\ 
& 𝐚_2^1 & \cdots & 𝐚_n^1   \end{bmatrix}
$$
where $𝐚_j^1 := (L_1 𝐚_j)[2:m]$ and $u_{1j} = a_{1j}$. But now consider
$$
L_2 := \begin{bmatrix} 1  \\ & L_{𝐚_2^1} \end{bmatrix}.
$$
Then
$$
L_2 L_1 A = \begin{bmatrix} u_{11} & u_{12} & \cdots & u_{1n} \\ 
    & u_{22} & u_{23} \cdots & u_{2n} \\
&& 𝐚_3^2 & \cdots & 𝐚_n^2   \end{bmatrix}
$$
where 
$$
r_{2j} :=  (𝐚_j^1)[1] \qquad \hbox{and} \qquad 𝐚_j^2 := (L_2 𝐚_j^1)[2:m-1]
$$
Thus the first two columns are triangular. 

The inductive construction is again clear. If we define $𝐚_j^0 := 𝐚_j$ we
have the construction
$$
\begin{align*}
L_j &:= \begin{bmatrix} I_{j-1 \times j-1}  \\ & L_{𝐚_j^j} \end{bmatrix} \\
𝐚_j^k &:= (L_k 𝐚_j^{k-1})[2:m-k+1]
 \\
u_{kj} &:= (L_k 𝐚_j^{k-1})[1]
\end{align*}
$$
Then
$$
L_n \cdots L_1 A = \underbrace{\begin{bmatrix} 
u_{11} & \cdots & u_{1n} \\ & \ddots & \vdots\\
                                        && u_{nn}\end{bmatrix}}_U
$$
i.e.
$$
A = \underbrace{L_n^{-1} \cdots L_1^{-1}}_L R.
$$



### PLU Decomposition


## 3. Cholesky Decomposition

Cholesky Decomposition is a form of Gaussian elimination (without pivoting)
that exploits symmetry in the problem. It is only relevant for _symmetric positive definite_
matrices.

**Definition (positive definite)** A square matrix $A \in \bbR^{n \times n}$ is _positive definite_ if
for all $𝐱 \in \bbR^n$ we have
$$
𝐱^\top A 𝐱 > 0
$$